# Stepwise Regression for Variable Selection

Explore stepwise regression as a method for variable selection

1. Run 1-parameter model for each dataset
2. Calculate WAIC

In [2]:
import rnaseq_lib3 as r
import pandas as pd
import pymc3 as pm

from collections import OrderedDict

/home/ubuntu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/ubuntu/anaconda3/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


# Inputs

In [4]:
# Read in centered data
df = pd.read_hdf('/mnt/data/expression/tcga_gtex_tpm_norm_filt.hd5')
# Subset
gtex = df[df.label == 'gtex'].sort_values('tissue')
normal = df[df.label == 'tcga-normal'].sort_values('tissue')
tumor = df[df.label == 'tcga-tumor'].sort_values('tissue')
# Read in drug genes
genes = df.columns[5:]
drug_genes = [x.split('\t')[0] for x in open('../data/druggable-genes.tsv', 'r').readlines()]
drug_genes = [x for x in drug_genes if x in df.columns]

In [11]:
sample = tumor.loc['TCGA-FY-A3I4-01']
print(f"Sample comes from tissue: {sample.tissue}")

Sample comes from tissue: Thyroid


In [4]:
training_genes = r.outlier.select_k_best_genes(normal, genes, 'tissue')

# Test — one parameter

In [5]:
thyroid = normal[normal.tissue == 'Thyroid']

In [6]:
thyroid_model, thyroid_trace = r.outlier.run_model(sample, thyroid, training_genes, 'tissue')

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


Multiprocess sampling (4 chains in 4 jobs)
NUTS: [PSMB11-Thyroid, GBP7-Thyroid, SLC22A9-Thyroid, UROC1-Thyroid, CA5A-Thyroid, SLITRK1-Thyroid, C3P1-Thyroid, SLCO1B1-Thyroid, SLC10A1-Thyroid, SLC25A47-Thyroid, LECT2-Thyroid, APOF-Thyroid, F9-Thyroid, POU3F2-Thyroid, UGT2B10-Thyroid, SHISA7-Thyroid, CPN1-Thyroid, C9-Thyroid, UGT1A4-Thyroid, C8A-Thyroid, HAO1-Thyroid, SERPINA7-Thyroid, SERPINA10-Thyroid, HGFAC-Thyroid, APOA5-Thyroid, CYP11B1-Thyroid, AKR1C4-Thyroid, VSTM2B-Thyroid, OTOS-Thyroid, BAAT-Thyroid, APOC4-Thyroid, CYP2C8-Thyroid, KCNJ9-Thyroid, SLC22A2-Thyroid, TFR2-Thyroid, F2-Thyroid, HRG-Thyroid, ITIH1-Thyroid, TSHR-Thyroid, CACNG7-Thyroid, SERPINC1-Thyroid, CTD-2182N23.1-Thyroid, SFTPD-Thyroid, AHSG-Thyroid, NKX2-1-Thyroid, SFTA3-Thyroid, SFTPA1-Thyroid, APOA2-Thyroid, SFTPA2-Thyroid, SFTPC-Thyroid, eps, a]
Sampling 4 chains: 100%|██████████| 4000/4000 [01:00<00:00, 66.09draws/s] 


In [11]:
pm.waic(thyroid_trace, thyroid_model, progressbar=True)

2000it [02:47, 11.95it/s]


WAIC_r(WAIC=2.893472349476437, WAIC_se=45.06051298224947, p_WAIC=19.048133756112527, var_warn=1)

# Single Parameter Compare Test

`pm.compare` workaround
```
MODEL_NAME_MAP = {
    0: "Model1",
    1: "Model2"
}

comp_df = (pm.compare([trace1, trace2], 
                      [model1, model2])
             .rename(index=MODEL_NAME_MAP))
```

In [19]:
models, traces = [], []
d = {i: x for i, x in enumerate(normal.tissue.unique())}
for tissue in normal.tissue.unique():
    sub = normal[normal.tissue == tissue]
    model, trace = r.outlier.run_model(sample, sub, training_genes, 'tissue')
    models.append(model)
    traces.append(trace)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


Multiprocess sampling (4 chains in 4 jobs)
NUTS: [PSMB11-Adrenal, GBP7-Adrenal, SLC22A9-Adrenal, UROC1-Adrenal, CA5A-Adrenal, SLITRK1-Adrenal, C3P1-Adrenal, SLCO1B1-Adrenal, SLC10A1-Adrenal, SLC25A47-Adrenal, LECT2-Adrenal, APOF-Adrenal, F9-Adrenal, POU3F2-Adrenal, UGT2B10-Adrenal, SHISA7-Adrenal, CPN1-Adrenal, C9-Adrenal, UGT1A4-Adrenal, C8A-Adrenal, HAO1-Adrenal, SERPINA7-Adrenal, SERPINA10-Adrenal, HGFAC-Adrenal, APOA5-Adrenal, CYP11B1-Adrenal, AKR1C4-Adrenal, VSTM2B-Adrenal, OTOS-Adrenal, BAAT-Adrenal, APOC4-Adrenal, CYP2C8-Adrenal, KCNJ9-Adrenal, SLC22A2-Adrenal, TFR2-Adrenal, F2-Adrenal, HRG-Adrenal, ITIH1-Adrenal, TSHR-Adrenal, CACNG7-Adrenal, SERPINC1-Adrenal, CTD-2182N23.1-Adrenal, SFTPD-Adrenal, AHSG-Adrenal, NKX2-1-Adrenal, SFTA3-Adrenal, SFTPA1-Adrenal, APOA2-Adrenal, SFTPA2-Adrenal, SFTPC-Adrenal, eps, a]
Sampling 4 chains: 100%|██████████| 4000/4000 [00:26<00:00, 152.33draws/s]


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


Multiprocess sampling (4 chains in 4 jobs)
NUTS: [PSMB11-Bile, GBP7-Bile, SLC22A9-Bile, UROC1-Bile, CA5A-Bile, SLITRK1-Bile, C3P1-Bile, SLCO1B1-Bile, SLC10A1-Bile, SLC25A47-Bile, LECT2-Bile, APOF-Bile, F9-Bile, POU3F2-Bile, UGT2B10-Bile, SHISA7-Bile, CPN1-Bile, C9-Bile, UGT1A4-Bile, C8A-Bile, HAO1-Bile, SERPINA7-Bile, SERPINA10-Bile, HGFAC-Bile, APOA5-Bile, CYP11B1-Bile, AKR1C4-Bile, VSTM2B-Bile, OTOS-Bile, BAAT-Bile, APOC4-Bile, CYP2C8-Bile, KCNJ9-Bile, SLC22A2-Bile, TFR2-Bile, F2-Bile, HRG-Bile, ITIH1-Bile, TSHR-Bile, CACNG7-Bile, SERPINC1-Bile, CTD-2182N23.1-Bile, SFTPD-Bile, AHSG-Bile, NKX2-1-Bile, SFTA3-Bile, SFTPA1-Bile, APOA2-Bile, SFTPA2-Bile, SFTPC-Bile, eps, a]
Sampling 4 chains: 100%|██████████| 4000/4000 [00:39<00:00, 117.85draws/s]


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


...

Multiprocess sampling (4 chains in 4 jobs)
NUTS: [PSMB11-Thymus, GBP7-Thymus, SLC22A9-Thymus, UROC1-Thymus, CA5A-Thymus, SLITRK1-Thymus, C3P1-Thymus, SLCO1B1-Thymus, SLC10A1-Thymus, SLC25A47-Thymus, LECT2-Thymus, APOF-Thymus, F9-Thymus, POU3F2-Thymus, UGT2B10-Thymus, SHISA7-Thymus, CPN1-Thymus, C9-Thymus, UGT1A4-Thymus, C8A-Thymus, HAO1-Thymus, SERPINA7-Thymus, SERPINA10-Thymus, HGFAC-Thymus, APOA5-Thymus, CYP11B1-Thymus, AKR1C4-Thymus, VSTM2B-Thymus, OTOS-Thymus, BAAT-Thymus, APOC4-Thymus, CYP2C8-Thymus, KCNJ9-Thymus, SLC22A2-Thymus, TFR2-Thymus, F2-Thymus, HRG-Thymus, ITIH1-Thymus, TSHR-Thymus, CACNG7-Thymus, SERPINC1-Thymus, CTD-2182N23.1-Thymus, SFTPD-Thymus, AHSG-Thymus, NKX2-1-Thymus, SFTA3-Thymus, SFTPA1-Thymus, APOA2-Thymus, SFTPA2-Thymus, SFTPC-Thymus, eps, a]
Sampling 4 chains: 100%|██████████| 4000/4000 [00:31<00:00, 128.28draws/s]


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


Multiprocess sampling (4 chains in 4 jobs)
NUTS: [PSMB11-Thyroid, GBP7-Thyroid, SLC22A9-Thyroid, UROC1-Thyroid, CA5A-Thyroid, SLITRK1-Thyroid, C3P1-Thyroid, SLCO1B1-Thyroid, SLC10A1-Thyroid, SLC25A47-Thyroid, LECT2-Thyroid, APOF-Thyroid, F9-Thyroid, POU3F2-Thyroid, UGT2B10-Thyroid, SHISA7-Thyroid, CPN1-Thyroid, C9-Thyroid, UGT1A4-Thyroid, C8A-Thyroid, HAO1-Thyroid, SERPINA7-Thyroid, SERPINA10-Thyroid, HGFAC-Thyroid, APOA5-Thyroid, CYP11B1-Thyroid, AKR1C4-Thyroid, VSTM2B-Thyroid, OTOS-Thyroid, BAAT-Thyroid, APOC4-Thyroid, CYP2C8-Thyroid, KCNJ9-Thyroid, SLC22A2-Thyroid, TFR2-Thyroid, F2-Thyroid, HRG-Thyroid, ITIH1-Thyroid, TSHR-Thyroid, CACNG7-Thyroid, SERPINC1-Thyroid, CTD-2182N23.1-Thyroid, SFTPD-Thyroid, AHSG-Thyroid, NKX2-1-Thyroid, SFTA3-Thyroid, SFTPA1-Thyroid, APOA2-Thyroid, SFTPA2-Thyroid, SFTPC-Thyroid, eps, a]
Sampling 4 chains: 100%|██████████| 4000/4000 [01:05<00:00, 60.81draws/s] 


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


Multiprocess sampling (4 chains in 4 jobs)
NUTS: [PSMB11-Uterus, GBP7-Uterus, SLC22A9-Uterus, UROC1-Uterus, CA5A-Uterus, SLITRK1-Uterus, C3P1-Uterus, SLCO1B1-Uterus, SLC10A1-Uterus, SLC25A47-Uterus, LECT2-Uterus, APOF-Uterus, F9-Uterus, POU3F2-Uterus, UGT2B10-Uterus, SHISA7-Uterus, CPN1-Uterus, C9-Uterus, UGT1A4-Uterus, C8A-Uterus, HAO1-Uterus, SERPINA7-Uterus, SERPINA10-Uterus, HGFAC-Uterus, APOA5-Uterus, CYP11B1-Uterus, AKR1C4-Uterus, VSTM2B-Uterus, OTOS-Uterus, BAAT-Uterus, APOC4-Uterus, CYP2C8-Uterus, KCNJ9-Uterus, SLC22A2-Uterus, TFR2-Uterus, F2-Uterus, HRG-Uterus, ITIH1-Uterus, TSHR-Uterus, CACNG7-Uterus, SERPINC1-Uterus, CTD-2182N23.1-Uterus, SFTPD-Uterus, AHSG-Uterus, NKX2-1-Uterus, SFTA3-Uterus, SFTPA1-Uterus, APOA2-Uterus, SFTPA2-Uterus, SFTPC-Uterus, eps, a]
Sampling 4 chains: 100%|██████████| 4000/4000 [00:39<00:00, 100.98draws/s]


In [28]:
md = OrderedDict({x: y for x, y in zip(models, traces)})
comp = pm.compare(md).rename(index=d)

In [32]:
comp

,WAIC,pWAIC,dWAIC,weight,SE,dSE,var_warn
Thyroid,2.61,19.25,0,0.95,45.68,0,1
Prostate,164.4,9.85,161.79,0.05,37.18,55.23,1
Breast,169.14,8.43,166.53,0,36.59,52.91,1
Head,170.11,8.06,167.5,0,36.04,52.88,1
Thymus,171.72,8.42,169.11,0,34.06,52.81,1
Colon,172.41,7.34,169.8,0,35.59,54.15,1
Cervix,174.89,5.54,172.28,0,34.11,50.07,1
Bladder,175.23,6.46,172.62,0,34.11,50.79,1
Uterus,177.18,6.6,174.57,0,33.36,49.75,1
Stomach,177.36,7.42,174.75,0,33.82,50.58,1


# Forward Stepwise Regression Informed by Pairwise Distance

In [16]:
ranks = r.outlier.pairwise_distance_ranks(sample, gtex, genes, 'tissue')
ranks.head(5)

,Group,MedianRank
0,Thyroid,138.5
1,Lung,599.0
2,Breast,612.0
3,Minor,940.0
4,Bladder,1084.0


In [17]:
groups = [
    ['Thyroid'],
    ['Thyroid', 'Lung'],
    ['Thyroid', 'Lung', 'Breast'],
    ['Thyroid', 'Lung', 'Breast', 'Minor'],
    ['Thyroid', 'Lung', 'Breast', 'Minor', 'Bladder']
]

out_dir = '/mnt/research_serializations/Stepwise-Regression'

In [18]:
models, traces = [], []
for group in groups:
    training_group = gtex[gtex.tissue.isin(group)]
    m, t = r.outlier.run_model(sample, training_group, drug_genes)
    models.append(m)
    traces.append(t)

Building model


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


Multiprocess sampling (4 chains in 4 jobs)
NUTS: [PARP1-Thyroid, AURKC-Thyroid, AURKB-Thyroid, AURKA-Thyroid, MS4A1-Thyroid, BTK-Thyroid, HSP90B1-Thyroid, HSP90AB3P-Thyroid, HSP90AB1-Thyroid, HSP90AA1-Thyroid, MDM2-Thyroid, BCL6-Thyroid, BCL2-Thyroid, SMO-Thyroid, GLI1-Thyroid, PTCH1-Thyroid, MAP2K2-Thyroid, MAP2K1-Thyroid, ARAF-Thyroid, RAF1-Thyroid, BRAF-Thyroid, HRAS-Thyroid, KRAS-Thyroid, NRAS-Thyroid, DEPTOR-Thyroid, MLST8-Thyroid, RPTOR-Thyroid, MTOR-Thyroid, TSC2-Thyroid, TSC1-Thyroid, AKT3-Thyroid, AKT1-Thyroid, PIK3C3-Thyroid, PIK3C2G-Thyroid, PIK3C2B-Thyroid, PIK3C2A-Thyroid, PIK3R6-Thyroid, PIK3R5-Thyroid, PIK3R4-Thyroid, PIK3R3-Thyroid, PIK3R2-Thyroid, PIK3R1-Thyroid, PIK3CD-Thyroid, PIK3CG-Thyroid, PIK3CB-Thyroid, PIK3CA-Thyroid, CDK2-Thyroid, CCNE1-Thyroid, CCND3-Thyroid, CCND2-Thyroid, CCND1-Thyroid, CDK6-Thyroid, CDK4-Thyroid, TEK-Thyroid, CSF1R-Thyroid, KIT-Thyroid, KDR-Thyroid, FLT3-Thyroid, NTRK3-Thyroid, NTRK2-Thyroid, NTRK1-Thyroid, FGFR4-Thyroid, FGFR3-Thyroid, FG

Building model


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


...

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


Multiprocess sampling (4 chains in 4 jobs)
NUTS: [PARP1-Thyroid, PARP1-Minor, PARP1-Lung, PARP1-Breast, AURKC-Thyroid, AURKC-Minor, AURKC-Lung, AURKC-Breast, AURKB-Thyroid, AURKB-Minor, AURKB-Lung, AURKB-Breast, AURKA-Thyroid, AURKA-Minor, AURKA-Lung, AURKA-Breast, MS4A1-Thyroid, MS4A1-Minor, MS4A1-Lung, MS4A1-Breast, BTK-Thyroid, BTK-Minor, BTK-Lung, BTK-Breast, HSP90B1-Thyroid, HSP90B1-Minor, HSP90B1-Lung, HSP90B1-Breast, HSP90AB3P-Thyroid, HSP90AB3P-Minor, HSP90AB3P-Lung, HSP90AB3P-Breast, HSP90AB1-Thyroid, HSP90AB1-Minor, HSP90AB1-Lung, HSP90AB1-Breast, HSP90AA1-Thyroid, HSP90AA1-Minor, HSP90AA1-Lung, HSP90AA1-Breast, MDM2-Thyroid, MDM2-Minor, MDM2-Lung, MDM2-Breast, BCL6-Thyroid, BCL6-Minor, BCL6-Lung, BCL6-Breast, BCL2-Thyroid, BCL2-Minor, BCL2-Lung, BCL2-Breast, SMO-Thyroid, SMO-Minor, SMO-Lung, SMO-Breast, GLI1-Thyroid, GLI1-Minor, GLI1-Lung, GLI1-Breast, PTCH1-Thyroid, PTCH1-Minor, PTCH1-Lung, PTCH1-Breast, MAP2K2-Thyroid, MAP2K2-Minor, MAP2K2-Lung, MAP2K2-Breast, MAP2K1-Thyro

Building model


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


INFO (theano.gof.compilelock): Waiting for existing lock by process '21281' (I am process '16555')
INFO (theano.gof.compilelock): To manually release the lock, delete /home/ubuntu/.theano/compiledir_Linux-4.15--generic-x86_64-with-debian-buster-sid-x86_64-3.6.5-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '21281' (I am process '16555')
INFO (theano.gof.compilelock): To manually release the lock, delete /home/ubuntu/.theano/compiledir_Linux-4.15--generic-x86_64-with-debian-buster-sid-x86_64-3.6.5-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '21281' (I am process '16555')
INFO (theano.gof.compilelock): To manually release the lock, delete /home/ubuntu/.theano/compiledir_Linux-4.15--generic-x86_64-with-debian-buster-sid-x86_64-3.6.5-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '21281' (I am process '16555')
INFO (theano.gof.compilelock): To manually release the lock, delete /home/ubunt

In [19]:
md = OrderedDict({x: y for x, y in zip(models, traces)})
comp = pm.compare(md)

INFO (theano.gof.compilelock): Waiting for existing lock by process '21281' (I am process '16555')
INFO (theano.gof.compilelock): To manually release the lock, delete /home/ubuntu/.theano/compiledir_Linux-4.15--generic-x86_64-with-debian-buster-sid-x86_64-3.6.5-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '21281' (I am process '16555')
INFO (theano.gof.compilelock): To manually release the lock, delete /home/ubuntu/.theano/compiledir_Linux-4.15--generic-x86_64-with-debian-buster-sid-x86_64-3.6.5-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '21281' (I am process '16555')
INFO (theano.gof.compilelock): To manually release the lock, delete /home/ubuntu/.theano/compiledir_Linux-4.15--generic-x86_64-with-debian-buster-sid-x86_64-3.6.5-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '21281' (I am process '16555')
INFO (theano.gof.compilelock): To manually release the lock, delete /home/ubunt

INFO (theano.gof.compilelock): Waiting for existing lock by process '21281' (I am process '16555')
INFO (theano.gof.compilelock): To manually release the lock, delete /home/ubuntu/.theano/compiledir_Linux-4.15--generic-x86_64-with-debian-buster-sid-x86_64-3.6.5-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '21281' (I am process '16555')
INFO (theano.gof.compilelock): To manually release the lock, delete /home/ubuntu/.theano/compiledir_Linux-4.15--generic-x86_64-with-debian-buster-sid-x86_64-3.6.5-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '21281' (I am process '16555')
INFO (theano.gof.compilelock): To manually release the lock, delete /home/ubuntu/.theano/compiledir_Linux-4.15--generic-x86_64-with-debian-buster-sid-x86_64-3.6.5-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '21281' (I am process '16555')
INFO (theano.gof.compilelock): To manually release the lock, delete /home/ubunt

INFO (theano.gof.compilelock): Waiting for existing lock by process '21281' (I am process '16555')
INFO (theano.gof.compilelock): To manually release the lock, delete /home/ubuntu/.theano/compiledir_Linux-4.15--generic-x86_64-with-debian-buster-sid-x86_64-3.6.5-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '21281' (I am process '16555')
INFO (theano.gof.compilelock): To manually release the lock, delete /home/ubuntu/.theano/compiledir_Linux-4.15--generic-x86_64-with-debian-buster-sid-x86_64-3.6.5-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '21281' (I am process '16555')
INFO (theano.gof.compilelock): To manually release the lock, delete /home/ubuntu/.theano/compiledir_Linux-4.15--generic-x86_64-with-debian-buster-sid-x86_64-3.6.5-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '21281' (I am process '16555')
INFO (theano.gof.compilelock): To manually release the lock, delete /home/ubunt

INFO (theano.gof.compilelock): Waiting for existing lock by process '21281' (I am process '16555')
INFO (theano.gof.compilelock): To manually release the lock, delete /home/ubuntu/.theano/compiledir_Linux-4.15--generic-x86_64-with-debian-buster-sid-x86_64-3.6.5-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '21281' (I am process '16555')
INFO (theano.gof.compilelock): To manually release the lock, delete /home/ubuntu/.theano/compiledir_Linux-4.15--generic-x86_64-with-debian-buster-sid-x86_64-3.6.5-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '21281' (I am process '16555')
INFO (theano.gof.compilelock): To manually release the lock, delete /home/ubuntu/.theano/compiledir_Linux-4.15--generic-x86_64-with-debian-buster-sid-x86_64-3.6.5-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '21281' (I am process '16555')
INFO (theano.gof.compilelock): To manually release the lock, delete /home/ubunt

In [20]:
comp

,WAIC,pWAIC,dWAIC,weight,SE,dSE,var_warn
3,196.42,30.81,0,0.54,20.46,0,1
0,196.56,41.9,0.14,0.46,19.93,6.46,1
4,199.1,29.73,2.68,0,20.34,1.02,1
1,200.92,39.37,4.5,0,19.86,5.56,1
2,202.01,36.76,5.59,0,19.87,4.7,1
